# Transformer 多模态医学 AI 教程（入门版）

本教程面向**完全新手**，一步一步带你完成一个可运行的**多模态医学 AI**小项目：

- **图像模态**（模拟医学影像）
- **文本模态**（模拟病历/报告）
- **Transformer** 作为核心编码器

我们将完成：
1. 数据准备（含自己的数据如何组织）
2. 构建多模态 Transformer 模型
3. 训练与评估
4. 推理与保存
5. 如何扩展到真实数据、预训练与微调


## 0. 环境准备

本教程需要 Python 3.9+，以及以下依赖：
- `torch`（PyTorch）
- `numpy`

如果你还没有安装 PyTorch，可以运行：

```bash
pip install torch --index-url https://download.pytorch.org/whl/cpu
```

> 说明：此教程使用 CPU 即可运行，速度足够演示。


In [ ]:
# 0.1 导入依赖
import os
import random
from dataclasses import dataclass

import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader


In [ ]:
# 0.2 固定随机种子，保证结果可复现
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

set_seed(42)


## 1. 数据准备

为了让教程**真正可运行**，我们先生成一个**可控的模拟数据集**。
你之后可以用真实医学数据替换（后面会详细讲如何组织你的数据）。

### 1.1 模拟数据说明
- **图像模态**：32×32 的灰度图，模拟医学影像
- **文本模态**：简短的中文病历描述
- **标签**：二分类（0=正常，1=异常）

我们人为制造一点规律，让模型能学到：
- 标签=1 时，图像中心有“亮点”
- 标签=1 时，文本包含更高概率的“异常词”


In [ ]:
# 1.2 构造一个简单的词表
vocab = [
    '[PAD]', '[UNK]',
    '咳嗽', '发热', '胸痛', '气促',
    '无明显异常', '影像提示结节', '影像提示炎症', '症状轻微',
    '血氧下降', '白细胞升高'
]
word2id = {w: i for i, w in enumerate(vocab)}
id2word = {i: w for w, i in word2id.items()}

PAD_ID = word2id['[PAD]']
UNK_ID = word2id['[UNK]']


In [ ]:
# 1.3 生成模拟数据
def generate_synthetic_sample(max_len=6):
    # 返回：image (1, 32, 32), text_ids (max_len), label (0/1)
    label = np.random.randint(0, 2)

    # --- 图像模态 ---
    image = np.random.rand(32, 32) * 0.1
    if label == 1:
        # 在中心区域加亮点，制造可学习模式
        image[12:20, 12:20] += 0.8
    image = np.clip(image, 0, 1)
    image = image.astype(np.float32)[None, :, :]  # (1, 32, 32)

    # --- 文本模态 ---
    normal_words = ['无明显异常', '症状轻微']
    abnormal_words = ['影像提示结节', '影像提示炎症', '血氧下降', '白细胞升高']

    words = []
    if label == 1:
        words += random.sample(abnormal_words, k=2)
    else:
        words += random.sample(normal_words, k=1)

    # 加一些常见症状词
    words += random.sample(['咳嗽', '发热', '胸痛', '气促'], k=2)

    # 组装并填充
    words = words[:max_len]
    text_ids = [word2id.get(w, UNK_ID) for w in words]
    text_ids += [PAD_ID] * (max_len - len(text_ids))

    return image, np.array(text_ids, dtype=np.int64), label


In [ ]:
# 1.4 构建 PyTorch Dataset
class SyntheticMedDataset(Dataset):
    def __init__(self, size=200, max_len=6):
        self.samples = [generate_synthetic_sample(max_len=max_len) for _ in range(size)]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image, text_ids, label = self.samples[idx]
        return (
            torch.tensor(image),
            torch.tensor(text_ids),
            torch.tensor(label, dtype=torch.long),
        )

# 划分训练/验证集
train_dataset = SyntheticMedDataset(size=200, max_len=6)
val_dataset = SyntheticMedDataset(size=60, max_len=6)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


## 1.5 如果你用自己的真实数据

推荐一个简单清晰的目录结构：

```
your_dataset/
  images/
    0001.png
    0002.png
  reports.csv
```

`reports.csv` 示例格式：

| image_id | text | label |
|----------|------|-------|
| 0001.png | 患者出现咳嗽、发热 | 1 |
| 0002.png | 无明显异常 | 0 |

然后你可以：
1. 用 `pandas` 读取 `reports.csv`
2. 用 `PIL` 或 `opencv` 读取图片
3. 把文本分词成 token（可以用简单空格切分，或使用 `jieba`）
4. 生成 `Dataset` 对象

> 后面会给你“真实数据替换模板”。


## 2. 构建多模态 Transformer 模型

我们将构建两个 Transformer 编码器：
- **文本 Transformer**：编码病历文本
- **图像 Transformer**：编码医学影像（简化版 ViT）

最后把两个模态的 `CLS` 向量拼接，进行分类。


In [ ]:
# 2.1 文本编码器
class TextTransformer(nn.Module):
    def __init__(self, vocab_size, d_model=64, nhead=4, num_layers=2, max_len=6):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_embedding = nn.Parameter(torch.randn(1, max_len, d_model))
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, batch_first=True)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.cls_token = nn.Parameter(torch.randn(1, 1, d_model))

    def forward(self, input_ids):
        # input_ids: (B, L)
        B, L = input_ids.shape
        x = self.embedding(input_ids) + self.pos_embedding[:, :L, :]
        cls = self.cls_token.expand(B, -1, -1)
        x = torch.cat([cls, x], dim=1)  # (B, L+1, d_model)
        x = self.encoder(x)
        return x[:, 0, :]  # 返回 CLS 向量


In [ ]:
# 2.2 图像编码器（简化版 ViT）
class ImageTransformer(nn.Module):
    def __init__(self, d_model=64, nhead=4, num_layers=2, image_size=32, patch_size=4):
        super().__init__()
        self.patch_size = patch_size
        num_patches = (image_size // patch_size) ** 2

        self.patch_embed = nn.Conv2d(1, d_model, kernel_size=patch_size, stride=patch_size)
        self.cls_token = nn.Parameter(torch.randn(1, 1, d_model))
        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, d_model))

        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, batch_first=True)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

    def forward(self, images):
        # images: (B, 1, 32, 32)
        x = self.patch_embed(images)  # (B, d_model, 8, 8)
        x = x.flatten(2).transpose(1, 2)  # (B, num_patches, d_model)
        B = x.size(0)
        cls = self.cls_token.expand(B, -1, -1)
        x = torch.cat([cls, x], dim=1)
        x = x + self.pos_embedding
        x = self.encoder(x)
        return x[:, 0, :]  # CLS


In [ ]:
# 2.3 多模态融合分类器
class MultimodalClassifier(nn.Module):
    def __init__(self, vocab_size, num_classes=2, d_model=64):
        super().__init__()
        self.text_encoder = TextTransformer(vocab_size=vocab_size, d_model=d_model)
        self.image_encoder = ImageTransformer(d_model=d_model)
        self.classifier = nn.Sequential(
            nn.Linear(d_model * 2, d_model),
            nn.ReLU(),
            nn.Linear(d_model, num_classes),
        )

    def forward(self, images, input_ids):
        text_feat = self.text_encoder(input_ids)
        image_feat = self.image_encoder(images)
        fused = torch.cat([text_feat, image_feat], dim=1)
        logits = self.classifier(fused)
        return logits


In [ ]:
# 2.4 初始化模型
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MultimodalClassifier(vocab_size=len(vocab)).to(device)
model


## 3. 训练模型

我们使用交叉熵损失（分类任务），并进行简单训练。
为了教程速度，这里训练 5 个 epoch。


In [ ]:
# 3.1 训练配置
epochs = 5
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

# 3.2 训练循环
for epoch in range(1, epochs + 1):
    model.train()
    total_loss = 0
    for images, input_ids, labels in train_loader:
        images = images.to(device)
        input_ids = input_ids.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        logits = model(images, input_ids)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch} | Train Loss: {avg_loss:.4f}')


## 4. 评估模型

我们计算验证集准确率。


In [ ]:
# 4.1 验证集评估
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, input_ids, labels in val_loader:
        images = images.to(device)
        input_ids = input_ids.to(device)
        labels = labels.to(device)

        logits = model(images, input_ids)
        preds = logits.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

acc = correct / total
print(f'Validation Accuracy: {acc:.2%}')


## 5. 推理（预测单个样本）

我们从验证集拿一个样本，看看模型的预测。


In [ ]:
# 5.1 拿一个样本进行推理
sample_image, sample_text, sample_label = val_dataset[0]

model.eval()
with torch.no_grad():
    logits = model(sample_image.unsqueeze(0).to(device), sample_text.unsqueeze(0).to(device))
    pred = logits.argmax(dim=1).item()

print('真实标签:', sample_label.item())
print('预测标签:', pred)


## 6. 如何保存与加载模型

训练完成后，我们可以保存模型权重，之后继续使用。


In [ ]:
# 6.1 保存模型
save_path = 'multimodal_transformer.pth'
torch.save(model.state_dict(), save_path)
print('模型已保存到:', save_path)

# 6.2 加载模型（示例）
loaded_model = MultimodalClassifier(vocab_size=len(vocab)).to(device)
loaded_model.load_state_dict(torch.load(save_path, map_location=device))
loaded_model.eval()
print('模型已加载，可继续推理')


## 7. 用真实数据的替换模板（非常重要）

下面给出一个“真实数据替换模板”，你只需要把自己的路径填进去即可。

### 7.1 数据结构回顾
```
your_dataset/
  images/
    0001.png
    0002.png
  reports.csv
```
### 7.2 读取真实数据的代码模板
（注意：此段代码是模板，需要你安装 `pandas` 和 `Pillow`）


In [ ]:
# === 真实数据模板示例（可复制替换）===
# import pandas as pd
# from PIL import Image
#
# class RealMedDataset(Dataset):
#     def __init__(self, csv_path, image_dir, tokenizer, max_len=64):
#         self.df = pd.read_csv(csv_path)
#         self.image_dir = image_dir
#         self.tokenizer = tokenizer  # 你可以自己写一个简单分词器
#         self.max_len = max_len
#
#     def __len__(self):
#         return len(self.df)
#
#     def __getitem__(self, idx):
#         row = self.df.iloc[idx]
#         image_path = os.path.join(self.image_dir, row['image_id'])
#         image = Image.open(image_path).convert('L').resize((32, 32))
#         image = np.array(image, dtype=np.float32) / 255.0
#         image = torch.tensor(image)[None, :, :]
#
#         text_ids = self.tokenizer(row['text'], max_len=self.max_len)
#         label = torch.tensor(row['label'], dtype=torch.long)
#
#         return image, text_ids, label
#
# # 使用方式：
# train_dataset = RealMedDataset('your_dataset/reports.csv', 'your_dataset/images', tokenizer)
# train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)


## 8. 预训练与微调（新手也能理解的版本）

### 8.1 预训练是什么？
- **预训练**：在大量数据上学习通用表示
- **微调**：在你的具体任务上继续训练

### 8.2 简化版建议
如果你是新手，可以这样做：
1. 先用本教程的合成数据训练通模型流程
2. 再把数据换成你自己的真实数据
3. 训练时使用较小学习率（如 1e-4）

### 8.3 如何做多模态预训练？
这里给你一个简单方向：
- **图像端预训练**：使用公开医学影像数据集（如胸片）做分类/自监督
- **文本端预训练**：使用大量医学报告做 Masked Language Model
- **融合层微调**：把两个编码器接起来，在你的任务上训练

> 真实预训练会比较复杂，但流程和本教程是一样的。


## 9. 小结

你已经完成了一个**可运行的多模态 Transformer 医学 AI 入门项目**：
- 数据准备
- 模型搭建
- 训练与评估
- 推理与保存
- 真实数据替换与预训练建议

如果你想继续拓展，可以尝试：
- 增大图像尺寸与模型规模
- 使用更真实的医学文本语料
- 引入对比学习或多模态预训练

祝你学习顺利！
